In [ ]:
# S-DES Implementation in Python

# Permutation functions
def permute(bits, pattern):
    return [bits[i - 1] for i in pattern]

# Left shift function
def left_shift(bits, n):
    return bits[n:] + bits[:n]

# Convert string to list of bits
def str_to_bits(s):
    return [int(bit) for bit in s]

# Convert list of bits to string
def bits_to_str(bits):
    return ''.join(str(bit) for bit in bits)

# S-Boxes
S0 = [[1, 0, 3, 2],
      [3, 2, 1, 0],
      [0, 2, 1, 3],
      [3, 1, 3, 2]]

S1 = [[0, 1, 2, 3],
      [2, 0, 1, 3],
      [3, 0, 1, 0],
      [2, 1, 0, 3]]

def s_box(bits, box):
    row = (bits[0] << 1) + bits[3]
    col = (bits[1] << 1) + bits[2]
    val = box[row][col]
    return [val >> 1, val & 1]

def fk(bits, subkey):
    EP = [4,1,2,3,2,3,4,1]
    P4 = [2, 4, 3, 1]
    
    left, right = bits[:4], bits[4:]
    expanded = permute(right, EP)
    xor_result = [b ^ k for b, k in zip(expanded, subkey)]
    
    left_half, right_half = xor_result[:4], xor_result[4:]
    sbox_output = s_box(left_half, S0) + s_box(right_half, S1)
    p4 = permute(sbox_output, P4)
    
    left_result = [l ^ p for l, p in zip(left, p4)]
    return left_result + right

def switch(bits):
    return bits[4:] + bits[:4]

def generate_keys(key_bits):
    P10 = [3, 5, 2, 7, 4, 10, 1, 9, 8, 6]
    P8 = [6, 3, 7, 4, 8, 5, 10, 9]

    permuted = permute(key_bits, P10)
    left, right = permuted[:5], permuted[5:]
    left1, right1 = left_shift(left, 1), left_shift(right, 1)
    k1 = permute(left1 + right1, P8)

    left2, right2 = left_shift(left1, 2), left_shift(right1, 2)
    k2 = permute(left2 + right2, P8)

    return k1, k2

def encrypt(plaintext, key):
    IP = [2, 6, 3, 1, 4, 8, 5, 7]
    IP_inv = [4, 1, 3, 5, 7, 2, 8, 6]

    key_bits = str_to_bits(key)
    pt_bits = str_to_bits(plaintext)
    k1, k2 = generate_keys(key_bits)
    permuted = permute(pt_bits, IP)
    round1 = fk(permuted, k1)
    swapped = switch(round1)
    round2 = fk(swapped, k2)
    ciphertext = permute(round2, IP_inv)
    return bits_to_str(ciphertext)

def decrypt(ciphertext, key):
    IP = [2, 6, 3, 1, 4, 8, 5, 7]
    IP_inv = [4, 1, 3, 5, 7, 2, 8, 6]

    key_bits = str_to_bits(key)
    ct_bits = str_to_bits(ciphertext)
    k1, k2 = generate_keys(key_bits)
    permuted = permute(ct_bits, IP)
    round1 = fk(permuted, k2)
    swapped = switch(round1)
    round2 = fk(swapped, k1)
    plaintext = permute(round2, IP_inv)
    return bits_to_str(plaintext)
plaintext = '10101010'
key = '1010000010'
cipher = encrypt(plaintext, key)
print("Encrypted:", cipher)
decrypted = decrypt(cipher, key)
print("Decrypted:", decrypted)

In [4]:
plaintext = '10101010'
key = '1010000010'

cipher = encrypt(plaintext, key)
print("Encrypted:", cipher)

decrypted = decrypt(cipher, key)
print("Decrypted:", decrypted)


Encrypted: 10001101
Decrypted: 10101010
